# introduce all variables

p^x_v - the probability of honest server outputting v
p^y_{a,g,b} - the probability of honest client choosing a, and outputting b given OT output g
aux^x_b - an auxiliary probability vector contributed by input x during simulation when TP returns b
(the cumulative probability for b=0 and b=1 is the same - some p_x)

Introduce all constraints - having proper distributions, mal. correctness, mal. privacy, and honest correctness. All requirements but honest correctness form an LP. The latter forms a combinatorial problem of finding all maximal sets in a graph (currently), in a certain set of graphs.
As a first step, lets rule out some small parameter values, such as l=1.
- Clearly, l=1 is not possible.
- We can quite easily check for l=2. l=3 gets chellanging, but still doable. Plans for going over l=3:
Optimizations that always work:
- Observation 1: The a's used by different columns must not intersect.
- Observation 2: Wlog. all coordinates are covered by some q_i. Assume the contrary. Then in all coordinates where only
one is covered, for all other q_i's sample a random entry at this coordinate instead. Then, any adversary in the new setting can be simulated
by a randomized adversary in the old setting picking a proper distribution (0.5, 0.5) (0,1) or (1,0). This yields a protocol covering all, which is secure.
- Observation 3: Due to symmetry, the labeling of the columns and rows is not crucial. There are several other symmetries that may be imployed. E.g for l=2, there are only 3 distinct classes of sets (A_1,A_2,A_3) to consider.




In [1]:
from sage.all import *
import itertools
import random
from sage.sat.solvers.satsolver import SAT
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from sage.misc.temporary_file import atomic_write
import copy
import time

In [2]:
solver = SAT(solver="LP")
solver.add_clause((-1,2))
solver.add_clause((1,3))
solution = solver()
print ' solution =',solution

 solution = [None, False, False, True]


In [ ]:
# col_sets is a set of column sets.
# TODO: apply symmetries to only test one representative in a class

# (r,   r+a)
# (r+a+c, b) 
l2solution = [((0,1,1,0),0), ((1,0,0,0),0), ((0,0,0,1),1), ((1,1,1,1),1), ((0,1,0,0),2), ((1,0,1,0),2)]

def validate_colvars(col_sets, l):
    if len(col_sets) < l:
        return false
    for i in range(l):
        if len(col_sets[i]) == 0:
            return false
    return true

def index_2_var(num, l):
    vals = []
    num = num - 1
    row_num = floor(num/pow(2,2*l))
    num = num % pow(2, 2*l)
    for i in range(2*l):
        cur_bit = num % 2
        vals.append(cur_bit)
        num = (num - cur_bit) / 2
    return (row_num, vals)    
        
# the tuple is for the form (v_1,0, v_1,1, v_2,0, v_2,1, v_3,0, v_3,1, .. v_l,1)
# row_num is in range(3)
def var_2_index(tup, row_num):
        n = 1
        p = 1
        l = len(tup)
        for i in range(l):
            n = n + p*tup[i]
            p = p*2
        return row_num*pow(2,l) + n    

def tuples_fixed_proj(l, proj_ind, proj_val):
    h = randint(1,4000)
    if (h == 5):
        print '==================== DEBUGGING ========== a,g = ', proj_ind, proj_val
    tuples = list()
    x = [0 for i in range(2*l)]
    for i in range(l):
        x[i*2 + proj_ind[i]] = proj_val[i]
    for g in itertools.product(range(2), repeat = l):
        for i in range(l):
            x[i*2 + 1 - proj_ind[i]] = g[i]
        if (h == 5):
            print 'modified x to ', x
        tuples.append(x[:])
    return tuples    
    
def honest_corr_some_priv(col_sets, l, forbidden_solutions):
    solver = SAT(solver="LP")
    print 'cols sets = ', col_sets
    for cl in forbidden_solutions:
        solver.add_clause(cl)
    print 'created solver, col_sets = ',col_sets
    # print 'adding correctness clauses ======================'
    psol = set([var_2_index(a, row_num) for (a,row_num) in l2solution])
    for col in range(3):
        # add ind. set clauses
        for g in itertools.product(range(2),repeat = l):
            oth1 = (col + 1) % 3
            oth2 = (col + 2) % 3
            rest = [oth1, oth2]
            A = col_sets[col]
            for a in A:
                cur_proj_set = tuples_fixed_proj(l, a, g)
                
                for oth in rest:
                    for cur_tup in cur_proj_set:
                        for oth_tup in cur_proj_set:
                            cur_clause =  (-var_2_index(cur_tup , col), -var_2_index(oth_tup , oth)) 
                            solver.add_clause(cur_clause)
                            # print 'added ',cur_clause, psol
                            clause_set = set([-x for x in cur_clause])
                            set_inter = psol.intersection(clause_set)
                            
    
    # print 'adding privacy clauses ======================'
    #add non-trivial-privacy constraints
    for col in range(3):
        A = col_sets[col]
        for g in itertools.product(range(2),repeat = l):
            row1 = (col + 1) % 3
            row2 = (col + 2) % 3
            # print 'cur_tuples = ',cur_tuples
            for a in A:
                cur_tuples = tuples_fixed_proj(l, a, g)
                for cur_tup in cur_tuples:
                    ind1 = -var_2_index(cur_tup, row1)
                    rest = [var_2_index(ind, row2) for ind in cur_tuples]
                    lind1 = tuple([ind1] + rest)
                    
                    solver.add_clause(lind1)
                    ind2 = -var_2_index(cur_tup, row2)
                    rest = [var_2_index(ind, row1) for ind in cur_tuples]                        
                    lind2 = tuple([ind2] + rest)
                    solver.add_clause(lind2)
                    
    # require that at least one v is used for each row
    # print 'adding non-triviality clauses'
    for row in range(3):
        cur_clause = [var_2_index(v,row) for v in itertools.product(range(2),repeat = 2*l)]
        solver.add_clause(cur_clause)
    
    print 'nvars = ',solver.nvars()
    out_val = solver()
    # print 'Output = ',out_val
    list_of_Ts = []
    forb_clause = []
    if out_val:
        for i, val in enumerate(out_val[1:]):
            if val == True: 
                list_of_Ts.append(index_2_var(i+1, l))
                forb_clause.append(-(i+1))
            else:
                forb_clause.append(i+1)
        # print 'list of T values is', list_of_Ts  
        print 'returning ', [-x for x in forb_clause if x < 0]  
    else:
        print 'There is no solution'
    return forb_clause

def find_all_solutions(col_sets, l):
    # for cols in col_sets:
    flag = True
    i = 1
    forbidden_solutions = []
    t0 = time.time()
    while flag:
        cur_solution = honest_corr_some_priv(col_sets, l, forbidden_solutions)
        if len(cur_solution) > 0:
            forbidden_solutions.append(tuple(cur_solution))
            print ' That was solution # ',i,' !!'
            i = i+1
        else:
            flag = False
    t1 = time.time()
    print 'it took ',t1 - t0

def get_l2_sets():
  #  col_sets = [({(0,0)}, {(1,1)}, {(0,1)}), ( {(0,0),(1,1)}, {(0,1)}, {(1,0)} ), ( {(0,0),(0,1)} , {(1,1)} , {(1,0)} )]
    col_sets = [{(0,0)}, {(1,1)}, {(1,0)}]
    find_all_solutions(col_sets, 2)
    
        
def iterate_sets(l):    
    indices = [index for index in itertools.product(range(2), repeat = l)]
    ndec = len(indices)
    x = 1
    for i in itertools.product(range(-1,3), repeat = ndec):
        cur_sets = [set(),set(),set()]
        for j in range(ndec):
            if i[j] > -1:
               cur_sets[i[j]].add(indices[j])   
        if validate_colvars(cur_sets, l):
            print 'step # ',x 
            find_all_solutions(cur_sets, l)
            x = x + 1    
                
# get_l2_sets()
iterate_sets(3)



step #  1
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [21, 45, 65, 121, 137, 138, 177]
 That was solution #  1  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 61, 85, 105, 157, 161]
 That was solution #  2  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 61, 62, 85, 105, 157, 161]
 That was solution #  3  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 61, 85, 105, 157, 161, 162]
 That was solution #  4  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)

returning  [1, 2, 11, 55, 61, 85, 86, 95, 99, 105, 151, 152, 157, 158, 161, 171]
 That was solution #  31  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 11, 55, 61, 62, 85, 86, 95, 99, 105, 151, 152, 157, 158, 161, 171]
 That was solution #  32  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 11, 55, 56, 61, 62, 85, 86, 95, 99, 105, 151, 152, 157, 158, 161, 162, 171]
 That was solution #  33  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 11, 55, 61, 62, 85, 86, 95, 96, 99, 105, 151, 152, 157, 158, 161, 162, 171]
 That was solution #  34  !!
cols sets =  [set([(1, 0, 1)]), set([(

returning  [1, 2, 11, 55, 61, 85, 95, 99, 105, 151, 157, 162, 171]
 That was solution #  61  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 11, 55, 61, 62, 85, 95, 99, 105, 152, 157, 162, 171]
 That was solution #  62  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 11, 55, 61, 85, 95, 96, 99, 105, 151, 152, 157, 162, 171]
 That was solution #  63  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 11, 55, 61, 85, 95, 99, 105, 151, 152, 157, 162, 171, 172]
 That was solution #  64  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set(

nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 83, 85, 86, 105, 107, 155, 156, 157, 158, 161, 162, 163]
 That was solution #  90  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 64, 83, 85, 86, 105, 107, 155, 156, 157, 158, 161, 162, 163, 164]
 That was solution #  91  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 64, 83, 85, 86, 105, 107, 155, 156, 157, 158, 161, 162, 163]
 That was solution #  92  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 11, 12, 55, 61, 62, 85, 86, 95, 96, 99, 100, 105, 151, 152, 157, 158, 162, 171, 172]
 That was sol

nvars =  192
returning  [1, 2, 7, 8, 61, 63, 64, 83, 85, 86, 105, 107, 155, 156, 157, 161, 162, 163]
 That was solution #  119  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 63, 64, 83, 84, 85, 86, 105, 107, 155, 156, 157, 161, 162, 163, 164]
 That was solution #  120  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 63, 64, 83, 84, 85, 86, 105, 107, 155, 156, 157, 161, 162, 163]
 That was solution #  121  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 63, 64, 83, 84, 85, 86, 105, 106, 107, 155, 156, 157, 161, 162, 163, 164]
 That was solution #  122 

...

created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 62, 63, 83, 85, 105, 107, 156, 157, 161, 163]
 That was solution #  597  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 62, 63, 83, 85, 105, 107, 156, 157, 161, 163, 164]
 That was solution #  598  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 62, 63, 83, 85, 105, 107, 156, 157, 161, 162, 163]
 That was solution #  599  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 62, 63, 83, 85, 105, 107, 156, 157, 158, 161, 162, 163]
 That was solution

nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 64, 83, 85, 86, 105, 107, 155, 156, 157, 158, 161, 164]
 That was solution #  626  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 64, 83, 84, 85, 86, 105, 107, 155, 156, 157, 158, 161, 162, 164]
 That was solution #  627  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 64, 83, 84, 85, 86, 105, 107, 155, 156, 157, 158, 161, 164]
 That was solution #  628  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 64, 83, 84, 85, 86, 105, 106, 107, 155, 156, 157, 158, 161, 162, 164]
 That was 

created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 64, 83, 84, 85, 86, 105, 107, 155, 157, 158, 161, 162, 163]
 That was solution #  655  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 64, 83, 84, 85, 86, 105, 106, 107, 155, 157, 158, 161, 162, 163, 164]
 That was solution #  656  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 8, 61, 62, 63, 64, 83, 84, 85, 86, 105, 106, 107, 155, 157, 158, 161, 162, 163]
 That was solution #  657  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7

created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 62, 63, 64, 83, 85, 86, 105, 107, 155, 156, 157, 158, 161, 162, 163, 164]
 That was solution #  683  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 62, 63, 64, 83, 85, 86, 105, 107, 155, 156, 157, 158, 161, 162, 163]
 That was solution #  684  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 62, 63, 64, 83, 84, 85, 86, 105, 107, 155, 156, 157, 158, 161, 162, 163, 164]
 That was solution #  685  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 6

created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 64, 83, 85, 105, 107, 155, 157, 161, 163]
 That was solution #  711  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 64, 83, 85, 105, 107, 155, 156, 157, 161, 163]
 That was solution #  712  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
created solver, col_sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
nvars =  192
returning  [1, 2, 7, 61, 64, 83, 85, 105, 107, 155, 157, 161, 163, 164]
 That was solution #  713  !!
cols sets =  [set([(1, 0, 1)]), set([(1, 1, 0)]), set([(1, 1, 1)])]
